In [19]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

#####################################################################
# Helper functions to make the code more readable.


def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    print('vec',vec)
    max_score = vec[0, argmax(vec)]
    print('max_score',max_score)
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    print('max_score_broadcast',max_score_broadcast)
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

#####################################################################
# Create model


class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        print('self.transitions111',self.transitions,tag_to_ix[START_TAG])
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        print('self.transitions222',self.transitions,tag_to_ix[STOP_TAG])

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        print('_forward_alg_feats',feats.size())
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                print('emit_score',emit_score.size())
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                print('next_tag_var',next_tag_var.size())
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        print('terminal_var',terminal_var.size())
        alpha = log_sum_exp(terminal_var)
        print('alpha',alpha)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        print('embeds',embeds.size())
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        print('lstm_out',lstm_out.size())
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        print('lstm_out222',lstm_out.size())
        lstm_feats = self.hidden2tag(lstm_out)
        print('lstm_feats',lstm_feats.size())
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        print('feats',feats.size())
        forward_score = self._forward_alg(feats)
        print('forward_score',forward_score.size())
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

#####################################################################
# Run training


START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print('first_output',model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        3):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print('last_test',model(precheck_sent))
# We got it!

self.transitions111 Parameter containing:
tensor([[-1.1811e-01, -1.4420e+00, -1.1108e+00, -1.1187e+00,  7.5800e-01],
        [-4.9566e-01, -1.9700e-01, -3.3396e-02,  1.4273e+00, -4.7282e-01],
        [-7.5307e-01, -4.3190e-01,  6.6930e-01,  6.5051e-01, -8.6923e-01],
        [-1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04],
        [ 1.8568e-01, -2.7636e-01, -5.9385e-01, -3.0606e-01,  6.7713e-01]],
       requires_grad=True) 3
self.transitions222 Parameter containing:
tensor([[-1.1811e-01, -1.4420e+00, -1.1108e+00, -1.1187e+00, -1.0000e+04],
        [-4.9566e-01, -1.9700e-01, -3.3396e-02,  1.4273e+00, -1.0000e+04],
        [-7.5307e-01, -4.3190e-01,  6.6930e-01,  6.5051e-01, -1.0000e+04],
        [-1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04],
        [ 1.8568e-01, -2.7636e-01, -5.9385e-01, -3.0606e-01, -1.0000e+04]],
       requires_grad=True) 4
embeds torch.Size([11, 1, 5])
lstm_out torch.Size([11, 1, 4])
lstm_out222 torch.Size([11, 4])
lstm_feats t

max_score_broadcast tensor([[7.1695, 7.1695, 7.1695, 7.1695, 7.1695]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ -9993.5957,  -9992.2383,  -9992.6533, -19991.5234,  -9992.5283]],
       grad_fn=<AddBackward0>)
max_score tensor(-9992.2383, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[-9992.2383, -9992.2383, -9992.2383, -9992.2383, -9992.2383]],
       grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 5.9038e+00,  6.7991e+00,  6.0671e+00, -9.9925e+03, -9.9932e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(6.7991, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[6.7991, 6.7991, 6.7991, 6.7991, 6.7991]], grad_fn=<ExpandBackward>)
terminal_var torch.Size([1, 5])
vec tensor([[ 6.7854e+00,  7.7912e+00,  7.0439e+00, -9.9916e+03, -9.9926e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(7.7912, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[7.7912,

max_score tensor(4.6819, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[4.6819, 4.6819, 4.6819, 4.6819, 4.6819]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 2.3757e+00,  4.0719e+00,  4.9030e+00, -9.9942e+03, -9.9966e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(4.9030, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[4.9030, 4.9030, 4.9030, 4.9030, 4.9030]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ -9996.0518,  -9994.6826,  -9994.9502, -19993.9980,  -9995.7861]],
       grad_fn=<AddBackward0>)
max_score tensor(-9994.6826, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[-9994.6826, -9994.6826, -9994.6826, -9994.6826, -9994.6826]],
       grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 3.0433e+00,  3.9466e+00,  3.3744e+00, -9.9954e+03, -9.9969e+03]],
       grad_fn=<AddBackward0>)
max_sc

max_score_broadcast tensor([[3.0670, 3.0670, 3.0670, 3.0670, 3.0670]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 3.0096e+00,  4.6261e+00,  4.4796e+00, -9.9931e+03, -9.9959e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(4.6261, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[4.6261, 4.6261, 4.6261, 4.6261, 4.6261]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 2.1497e+00,  3.8357e+00,  4.6466e+00, -9.9944e+03, -9.9965e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(4.6466, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[4.6466, 4.6466, 4.6466, 4.6466, 4.6466]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ -9996.3701,  -9995.0137,  -9995.3135, -19994.3184,  -9995.7812]],
       grad_fn=<AddBackward0>)
max_score tensor(-9995.0137, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[-999

next_tag_var torch.Size([1, 5])
vec tensor([[ -9999.1768,  -9997.5459,  -9998.0332, -19997.0137,  -9998.2012]],
       grad_fn=<AddBackward0>)
max_score tensor(-9997.5459, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[-9997.5459, -9997.5459, -9997.5459, -9997.5459, -9997.5459]],
       grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 1.6915e-01,  1.3186e+00,  5.4018e-01, -9.9982e+03, -9.9990e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(1.3186, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[1.3186, 1.3186, 1.3186, 1.3186, 1.3186]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 8.7876e-01,  9.8520e-01,  9.7560e-01, -9.9980e+03, -9.9983e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(0.9852, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[0.9852, 0.9852, 0.9852, 0.9852, 0.9852]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_t

emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[-8.7042e-01,  8.4680e-01,  1.5836e+00, -9.9974e+03, -9.9992e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(1.5836, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[1.5836, 1.5836, 1.5836, 1.5836, 1.5836]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ -9999.2432,  -9997.8633,  -9998.2451, -19997.2188,  -9998.3760]],
       grad_fn=<AddBackward0>)
max_score tensor(-9997.8633, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[-9997.8633, -9997.8633, -9997.8633, -9997.8633, -9997.8633]],
       grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 2.6341e-01,  1.1510e+00,  4.8018e-01, -9.9982e+03, -9.9990e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(1.1510, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[1.1510, 1.1510, 1.1510, 1.1510, 1.1510]], grad_fn=<ExpandBackward>)
emit_s

max_score tensor(6.4961, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[6.4961, 6.4961, 6.4961, 6.4961, 6.4961]], grad_fn=<ExpandBackward>)
terminal_var torch.Size([1, 5])
vec tensor([[ 6.6449e+00,  7.5157e+00,  6.9915e+00, -9.9917e+03, -9.9927e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(7.5157, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[7.5157, 7.5157, 7.5157, 7.5157, 7.5157]], grad_fn=<ExpandBackward>)
alpha tensor(8.2142, grad_fn=<AddBackward0>)
forward_score torch.Size([])
embeds torch.Size([7, 1, 5])
lstm_out torch.Size([7, 1, 4])
lstm_out222 torch.Size([7, 4])
lstm_feats torch.Size([7, 5])
feats torch.Size([7, 5])
_forward_alg_feats torch.Size([7, 5])
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[-1.0000e+04, -1.0002e+04, -1.0001e+04, -1.1670e+00, -2.0000e+04]],
       grad_fn=<AddBackward0>)
max_score tensor(-1.1670, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[-1.1670, -1.1670, -1.1670, -1.1670, -1.1670]],
  

vec tensor([[ 2.7864e+00,  4.1406e+00,  4.0196e+00, -9.9935e+03, -9.9965e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(4.1406, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[4.1406, 4.1406, 4.1406, 4.1406, 4.1406]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ 1.9600e+00,  3.4513e+00,  4.3160e+00, -9.9948e+03, -9.9970e+03]],
       grad_fn=<AddBackward0>)
max_score tensor(4.3160, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[4.3160, 4.3160, 4.3160, 4.3160, 4.3160]], grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec tensor([[ -9996.4160,  -9995.2686,  -9995.5156, -19994.5664,  -9996.1572]],
       grad_fn=<AddBackward0>)
max_score tensor(-9995.2686, grad_fn=<SelectBackward>)
max_score_broadcast tensor([[-9995.2686, -9995.2686, -9995.2686, -9995.2686, -9995.2686]],
       grad_fn=<ExpandBackward>)
emit_score torch.Size([1, 5])
next_tag_var torch.Size([1, 5])
vec te